In [ ]:
!pip3 install webcolors 
!pip3 install selenium
!pip3 install bs4
!pip3 install thecolorapi
'''

!pip3 install pinterest-api
!pip3 install oauth2
'''

In [ ]:

import numpy as np
from PIL import Image
import pandas as pd
from tqdm import tqdm
from scipy.spatial import KDTree
import webcolors
import os
import random
#from pinterest import Api
import requests, json
import subprocess
import sys
from base64 import b64encode
import urllib.request
from selenium import webdriver
from urllib.request import urlopen
import re
#import pinterest
import requests 
from bs4 import BeautifulSoup 
from datetime import datetime
import math
from thecolorapi import color as thecolorapi_color
import matplotlib.pyplot as plt
from skimage import io
import time

testurl = 'https://www.pinterest.com/miraculouswip/flowers/'



In [ ]:
#https://stackoverflow.com/questions/48225334/extracting-data-from-pinterest-using-beautifulsoup-python
#couldnt figure ^ out but could be useful

def getdata(url): 
    r = requests.get(url) 
    return r.text 
def checkURL(url):
  if 'www.pinterest.com/' in url:
    return True
  else:
    return False

def getImages(url):
  if checkURL(url):
    htmldata = getdata(url) 
    soup = BeautifulSoup(htmldata, 'html.parser')

    urlarr = []
    for item in soup.find_all('img'):
      urlarr.append(item['src'])
    return urlarr
  else:
    print("Invalid pinterest url")
def getImageFiles(url):
  skiplist = {'1e78a2658daa724a58773b2bdf98b68c.jpg', '37f894920d87d43da19cd5d9760e121c.jpg'}
  urlarr = getImages(url)
  foldername = f'pins_{datetime.now()}'
  os.mkdir(foldername)
  os.chdir(foldername)
  fnamearr = []
  for imageurl in urlarr:
    rev = imageurl[::-1]
    print(rev[::-1])
    filename = rev[:rev.find('/')]
    filename = filename[::-1]

    if filename in skiplist:
      continue
    fnamearr.append(filename)

    urllib.request.urlretrieve(imageurl, filename)

    img = Image.open(filename)

    img.show()
  os.chdir('..')
  return fnamearr, foldername







def randrgf_img(img_og, numsamples=1000, disp = False, mode = "RGB"):
  img_og = img_og.convert(mode)
  imgarr = np.array(img_og)
  cellarr = []
  for _ in range(numsamples):
    i = random.randint(0, len(imgarr)-1)
    j = random.randint(0, len(imgarr[0])-1)
    cell = imgarr[i][j]
    if tuple(cell) in colornamedict.keys():
        tup = colornamedict[tuple(cell)]
    else:
      tup = convert_rgb_to_names(tuple(cell)) 
      colornamedict[tuple(cell)] = tup
    r = cell[0]
    g = cell[1]
    b = cell[2]
    cellarr.append([i,j,r,g,b, tup[0], tup[1],huecalc(r,g,b),satcalc(r,g,b),lightness(r,g,b),rgb_val(r,g,b)])

  #print(cellarr)
  imgpd = pd.DataFrame(columns=["row", "column", "R", "G", "B", "name", "distance", "hue", "saturation", "lightness", "val"], data = cellarr)
  imgpd['distance'] /= 255
  if disp:
    print(imgpd.name.value_counts().head(10))
  #print(imgpd)
  return imgpd

def assemble_df_direct(url, numsamples = 1000,disp= False):
  #print("Assemble_direct")
  skiplist = {'1e78a2658daa724a58773b2bdf98b68c.jpg', '37f894920d87d43da19cd5d9760e121c.jpg'}
  urlarr = getImages(url)
  fnamearr = []
  df_arr = []
  for imageurl in urlarr:
    rev = imageurl[::-1]
    if disp:
      print(rev[::-1])
    filename = rev[:rev.find('/')]
    filename = filename[::-1]

    if filename in skiplist:
      continue
    fnamearr.append(filename)

    urllib.request.urlretrieve(imageurl, filename)

    img = Image.open(filename)

    df_arr.append(randrgf_img(img,numsamples=numsamples,disp = disp))#does HSL too
  

  returnpd = pd.concat(df_arr, sort = False)
  return returnpd

In [ ]:
def assemble_df(filearr, folder_name, numsamples=1000):
  os.chdir(f'/content/{folder_name}')
  dfarr = []
  for file in filearr:
    temppd = HSLanalyzer(file, numsamples)
    temppd.assign(fname=[file for i in range(len(temppd["R"]))])
    dfarr.append(temppd)

  returnpd = pd.concat(dfarr, sort = False)
  os.chdir('../')
  return returnpd

In [ ]:
def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = webcolors.CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(webcolors.hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return names[index], distance

def convert_hsl_to_rgb(hsl_tuple):
  hsl_tuple[0] = round(hsl_tuple[0])
  hsl_tuple[1] = round(100*hsl_tuple[1])
  hsl_tuple[2] = round(100*hsl_tuple[2])
  color = thecolorapi_color(hsl = tuple(hsl_tuple))
  return color.rbg
def convert_hsl_to_name(hsl_tuple):
  h = round(hsl_tuple[0])
  s = round(100*hsl_tuple[1])
  l = round(100*hsl_tuple[2])
  result = requests.get(f"https://www.thecolorapi.com/id?hsl={h},{s}%,{l}%&format=json")
  return result.json()["name"]["value"]


def convert_hsl_to_hex(hsl_tuple):
  h = round(hsl_tuple[0])
  s = round(100*hsl_tuple[1])
  l = round(100*hsl_tuple[2])
  result = requests.get(f"https://www.thecolorapi.com/id?hsl={h},{s}%,{l}%&format=json")
  return result.json()["hex"]["clean"]

def convert_hsl_to_rgb(hsl_tuple):
  h = round(hsl_tuple[0])
  s = round(100*hsl_tuple[1])
  l = round(100*hsl_tuple[2])
  result = requests.get(f"https://www.thecolorapi.com/id?hsl={h},{s}%,{l}%&format=json")
  rgb= result.json()["rgb"]
  return (rgb['r'], rgb['g'], rgb['b'])

def convert_hsl_to_json(hsl_tuple):
  h = round(hsl_tuple[0])
  s = round(100*hsl_tuple[1])
  l = round(100*hsl_tuple[2])
  result = requests.get(f"https://www.thecolorapi.com/id?hsl={h},{s}%,{l}%&format=json")
  return result.json()

In [ ]:
 
checkFile = "colornamedict.csv"
 
 
 
if os.path.isfile(checkFile):
  pathExists = True
  colornamedict = {}#CHANGE ONCE I FIGURE OUT HOW TO FORMAT COLORDICT CSV
else:
  pathExists = False
  colornamedict = {}

In [ ]:
def huecalc(r,g,b):
  r /= 255
  g /= 255
  b /= 255
  cmax = max(r,g,b)
  cmin = min(r,g,b)
  delta = cmax - cmin
  if delta == 0:
    return 0
  elif cmax == r:
    return 60*(((g-b)/delta)%6)
  elif cmax == g:
    return 60*(((b-r)/delta)+2)
  elif cmax == b:
    return 60*(((r-g)/delta)+4)
  else:
    return 0


def satcalc(r,g,b):
  lightcalc = lightness(r,g,b)
  r /= 255
  g /= 255
  b /= 255
  cmax = max(r,g,b)
  cmin = min(r,g,b)
  delta = cmax - cmin
  if delta == 0:
    return 0
  else:
    return (delta/(1-abs(2*lightcalc - 1)))

def lightness(r,g,b):
  r /= 255
  g /= 255
  b /= 255
  cmax = max(r,g,b)
  cmin = min(r,g,b)
  return ((cmax+cmin)/2)

def rgb_val(r,g,b):
  return (max(r,g,b)/255)

In [ ]:


def singlecell_rgf(imgpath = "/content/test.jpg"):
  mode = "RGB"
  img_og = Image.open(imgpath).convert(mode)
  imgarr = np.array(img_og)
  cellarr = []
  for i in tqdm (range (len(imgarr)),desc="Row..."):
    row = imgarr[i]
    for j, cell in enumerate(row):
      if tuple(cell) in colornamedict.keys():
        tup = colornamedict[tuple(cell)]
      else:
        tup = convert_rgb_to_names(tuple(cell))
        colornamedict[tuple(cell)] = tup
      cellarr.append([i,j,cell[0], cell[1], cell[2], tup[0], tup[1]])
  imgpd = pd.DataFrame(columns=["row","column", "R", "G", "B", "name", "distance"], data = cellarr)
  print(imgpd.name.value_counts().head(10))
  return imgpd


def multicell_rgf(stepsize= 5,imgpath = "/content/test.jpg"):
  mode = "RGB"
  img_og = Image.open(imgpath).convert(mode)
  imgarr = np.array(img_og)
  cellarr = []

  for i in tqdm (range (0, len(imgarr), stepsize),desc="Rows completed"):
    
    row = imgarr[i:min(i+stepsize, len(imgarr))]
    rl = len(row)
    #print(f"row[0]: {row[0]}")
    for j in range(0,len(row[0]),stepsize):
      #print(f"{i},{j}")
      cell = row[:,j:min(j+stepsize, len(imgarr[0]))]
      cl = len(cell)
      avgcell = [round(np.average(cell[:,:,0])), round(np.average(cell[:,:,1])),round(np.average(cell[:,:,2]))]
      if tuple(avgcell) in colornamedict.keys():
        tup = colornamedict[tuple(avgcell)]
      else:
        tup = convert_rgb_to_names(tuple(avgcell))
        colornamedict[tuple(avgcell)] = tup
      temp = []
      temp.append(i)
      temp.append(j)
      temp.append(rl)
      temp.append(cl)
      temp.append(avgcell[0])
      temp.append(avgcell[1])
      temp.append(avgcell[2])
      temp.append(tup[0])
      temp.append(tup[1])
      
      cellarr.append(temp)
  imgpd = pd.DataFrame(columns=["row(top left ind)","column(top left ind)", "rowlength", "collength", "R", "G", "B", "name", "distance"], data = cellarr)
  print(imgpd.name.value_counts().head(10))
  return imgpd



def randrgf(imgpath = "/content/test.jpg", numsamples=1000, disp = False):
  mode = "RGB"
  img_og = Image.open(imgpath).convert(mode)
  imgarr = np.array(img_og)
  cellarr = []
  for _ in range(numsamples):
    i = random.randint(0, len(imgarr)-1)
    j = random.randint(0, len(imgarr[0])-1)
    cell = imgarr[i][j]
    if tuple(cell) in colornamedict.keys():
        tup = colornamedict[tuple(cell)]
    else:
      tup = convert_rgb_to_names(tuple(cell)) 
      colornamedict[tuple(cell)] = tup
    
    cellarr.append([i,j,cell[0], cell[1], cell[2], tup[0], tup[1]])

  #print(cellarr)
  imgpd = pd.DataFrame(columns=["row", "column", "R", "G", "B", "name", "distance"], data = cellarr)

  if disp:
    print(imgpd.name.value_counts().head(10))
  #print(imgpd)
  return imgpd

def randrgf_img(img_og, numsamples=1000, disp = False, mode = "RGB"):
  img_og = img_og.convert(mode)
  imgarr = np.array(img_og)
  cellarr = []
  for _ in range(numsamples):
    i = random.randint(0, len(imgarr)-1)
    j = random.randint(0, len(imgarr[0])-1)
    cell = imgarr[i][j]
    if tuple(cell) in colornamedict.keys():
        tup = colornamedict[tuple(cell)]
    else:
      tup = convert_rgb_to_names(tuple(cell)) 
      colornamedict[tuple(cell)] = tup
    
    cellarr.append([i,j,cell[0], cell[1], cell[2], tup[0], tup[1]])

  #print(cellarr)
  imgpd = pd.DataFrame(columns=["row", "column", "R", "G", "B", "name", "distance"], data = cellarr)

  if disp:
    print(imgpd.name.value_counts().head(10))
  #print(imgpd)
  return imgpd


def sigmoid(x):
  
    z = np.exp(-x)
    sig = 1 / (1 + z)

    return sig

In [ ]:
def HSLanalyzer(imgpath = "/content/test.jpg", numsamples = 1000):
  testpd = randrgf(imgpath, numsamples)
  hue = []
  sat = []
  light = []
  val = []
  for index,row in testpd.iterrows():
    row = row.to_numpy()
    #print(row)
    r = int(row[2])
    g = int(row[3])
    b = int(row[4])
    if r < 0:
      r += 256
    if g < 0:
      g += 256
    if b < 0:
      b += 256
    #print(f"r: {r}")
    hue.append(huecalc(r,g,b))
    sat.append(satcalc(r,g,b))
    light.append(lightness(r,g,b))
    val.append(rgb_val(r,g,b))
    #print(row)
  #print(hue)
  testpd.insert(len(testpd.columns), "hue", hue)
  testpd.insert(len(testpd.columns), "saturation", sat)
  testpd.insert(len(testpd.columns), "lightness", light)
  testpd.insert(len(testpd.columns), "val", val)

  testpd['distance'] /= 255


  return testpd

In [ ]:
def curvecheck(sat, light):
  a = ((light/sat)-1)/(sat-1)
  b = 1-a

  xarr = np.asarray([0.1*i for i in range(10)])
  xarr += sat
  yarr = [a*x**2 + b*x for x in xarr]

  return xarr, yarr
  #double check this function, I ran into some issues with the curves going out of bounds at the boundary conditions

def hsl_distance(hsl_1, hsl_2):
  coord1 = (hsl_1[1]*np.cos(np.deg2rad(hsl_1[0])),hsl_1[1]*np.sin(np.deg2rad(hsl_1[0])), hsl_1[2])
  coord2 = (hsl_2[1]*np.cos(np.deg2rad(hsl_2[0])),hsl_2[1]*np.sin(np.deg2rad(hsl_2[0])), hsl_2[2])

  return math.sqrt((coord1[0]-coord2[0])**2+(coord1[1]-coord2[1])**2+(coord1[2]-coord2[2])**2)

In [ ]:
from operator import indexOf

def get_freqtable(colornames, color_df, color_quant = 0.2):
  freqtable = np.asarray([[name, len(color_df[color_df['name']==name]), round(color_df[color_df['name']==name]['hue'].mean(),3), round(color_df[color_df['name']==name]['saturation'].mean(),3), round(color_df[color_df['name']==name]['lightness'].mean(),3), round(color_df[color_df['name']==name]['distance'].mean(),4)] for name in colornames])
  freqtable = pd.DataFrame(columns = ["name", "freq","avg_hue","avg_saturation","avg_lightness", "avg_distance"], data= freqtable)
  freqtable["freq"]= pd.to_numeric(freqtable["freq"])
  freqtable.dropna()
  freqtable = freqtable[freqtable['freq'] != 0]
  freqtable["avg_hue"] = pd.to_numeric(freqtable["avg_hue"])
  freqtable["avg_saturation"]  = pd.to_numeric(freqtable["avg_saturation"])
  freqtable["avg_lightness"]= pd.to_numeric(freqtable["avg_lightness"])
  freqtable["avg_distance"]= pd.to_numeric(freqtable["avg_distance"])
  return freqtable

def swath_gen(h,xarr, yarr,rank, distrib):
  vals = [0,1,2,3,4,5,6,7,8,9]
  swatharr = tuple(zip(distrib, vals))
  xarr = list(xarr)
  yarr = list(yarr)
  if len(yarr) == 1:
    yarr = yarr[0]
  if len(xarr) == 1:
    xarr = xarr[0]
  keylist = sorted(swatharr, key = lambda x:x[0])
  top_n = 0
  if rank == "60":
    #print(swatharr)
    top_n = 3
    

  elif rank == "30":
    top_n = 2

  elif rank == "10":
    top_n = 1
  else:
    print(f"Invalid rank: {rank}")
    return []
  keylist = [swatharr[i][1] for i in range(top_n)]
  #print(f"xarr: {xarr}")
  #print(f"yarr: {yarr}")
  additional= [[h,xarr[i],yarr[i]] for i in keylist]
  return additional

def get_colordf(testcombo, color_quant = .1):
  colornames = testcombo['name'].unique()
  if testcombo["saturation"].mean() > 0.5:
    satbool = testcombo["saturation"] < testcombo["saturation"].quantile(1-color_quant)
  else:
    satbool = testcombo["saturation"] > testcombo["saturation"].quantile(color_quant)
  
  if testcombo["lightness"].mean() > 0.5:
    lightbool = testcombo["lightness"] < testcombo["lightness"].quantile(1-color_quant)
  else:
    lightbool = testcombo["lightness"] > testcombo["lightness"].quantile(color_quant)
  color_df = testcombo[lightbool & satbool]
  return colornames, color_df
def get_rankingdf(distance_df, color):
  ranking_df = distance_df.copy()
  ranking_df.drop("subtype", axis = 1, inplace = True)
  ranking_df = ranking_df[['prim_color','type', "score"]]
  ranking_df = ranking_df.groupby("type").aggregate(["sum"]).sort_values(('score','sum'), ascending = True)
  ranking_df.columns = [ "prim_color", "score"]
  ranking_df["prim_color"] = [color for _ in range(len(ranking_df))]
  ranking_df['score']/=ranking_df['score'].sum()
  self_score = ranking_df.iloc[0]["score"]
  #print(ranking_df.index)
  ranking_df=ranking_df.drop('Self')
  ranking_df['score']/=ranking_df['score'].sum()

  return ranking_df

def n_mostcommon(color_df, rgb_list, n):
  rgb_count = []
  for rgb in rgb_list:
    num_rgb = len(color_df[(abs(color_df["R"]-rgb[0]) < 5) & (abs(color_df["G"]-rgb[1]) < 5) & (abs(color_df["B"]-rgb[2]) < 0.05)])
    rgb_count.append([rgb, rgb_count])
  rgb_count = sorted(rgb_count, key = lambda x:x[1])
  return list(np.asarray(rgb_count)[:n,0])
def hue_distance_calc(h,xarr,yarr,df):
  lim = 400

  #print("hdc")
  #clock = ticktock()
  #clock.tick()
  hslarr = [[h, xarr[i],yarr[i]] for i in range(len(xarr))]
  distance = 0
  freqlist = [0,0,0,0,0,0,0,0,0,0]
  if len(df) < lim:
    for index,row in df.iterrows():
      temp_d = []
      
      row_hsl = [row["hue"],row["saturation"], row["lightness"]]
      for trip in hslarr:
        temp_d.append(hsl_distance(trip, row_hsl))
      distance += min(temp_d)
      freqlist[temp_d.index(min(temp_d))] += 1
    #clock.tock()
    #print("hdc end")
  else:
    for i in range(lim):
      index = random.randint(0, len(df)-1)
      row = df.iloc[index]
      
      temp_d = []
      
      row_hsl = [row["hue"],row["saturation"], row["lightness"]]
      for trip in hslarr:
        temp_d.append(hsl_distance(trip, row_hsl))
      distance += min(temp_d)
      freqlist[temp_d.index(min(temp_d))] += 1
    #clock.tock()
  return distance, np.asarray(freqlist)


In [ ]:

def get_pallette(color_df, color, freqtable, color_quant = 0.1, pal_limit = 0.9, palette_length = 6):
  
  
  clock = ticktock()

  
  color_hue = float(freqtable[freqtable['name'] == color]['avg_hue'])

  comphue = (color_hue-180)%360
  analagous_hue1 = (color_hue-30)%360
  analagous_hue2 = (color_hue+30)%360
  triadic_hue1 = (color_hue-120)%360
  triadic_hue2 = (color_hue+120)%360
  split_hue1 = (color_hue+150)%360
  split_hue2 = (color_hue-150)%360
  square_hue1 = (color_hue+90)%360
  square_hue2 = (color_hue-90)%360

  
  self_df = color_df[abs(color_df['hue']-color_hue) < 30]
  complementary_df =  color_df[abs(color_df['hue']-comphue) < 30]
  anal_df1 = color_df[abs(color_df['hue']-analagous_hue1) < 15]
  anal_df2 = color_df[abs(color_df['hue']-analagous_hue2) < 15]
  tri_df1 = color_df[abs(color_df['hue']-triadic_hue1) < 15]
  tri_df2 = color_df[abs(color_df['hue']-triadic_hue2) < 15]
  split_df1 = color_df[abs(color_df['hue']-split_hue1) < 15]
  split_df2 = color_df[abs(color_df['hue']-split_hue2) < 15]
  square_df1 = color_df[abs(color_df['hue']-comphue) < 20]
  square_df2 = color_df[abs(color_df['hue']-square_hue1) < 5]
  square_df3 = color_df[abs(color_df['hue']-square_hue2) < 5]
  tetra_df1 = color_df[abs(color_df['hue']-comphue) < 20] #used for both tetras
  tetra_df1_2 = color_df[abs(color_df['hue']-analagous_hue1) < 5]
  tetra_df1_3 = color_df[abs(color_df['hue']-split_hue1) < 5]
  tetra_df2_2 = color_df[abs(color_df['hue']-analagous_hue2) < 5]
  tetra_df2_3 = color_df[abs(color_df['hue']-split_hue2) < 5]



  self_count = len(self_df)
  complementary_count = len(complementary_df)
  anal_count1 =  len(anal_df1) 
  anal_count2 = len(anal_df2)
  triadic_count1 = len(tri_df1)
  triadic_count2  = len(tri_df2)

  split_count1 = len(split_df1)
  split_count2 = len(split_df2)

  square_count1 =  len(square_df1)
  square_count2 =  len(square_df2)
  square_count3 =  len(square_df3)

  tetradic_count1_1 = len(tetra_df1)
  tetradic_count1_2 = len(tetra_df1_2)
  tetradic_count1_3 = len(tetra_df1_3)
  tetradic_count2_1 = len(tetra_df1)
  tetradic_count2_2 = len(tetra_df2_2)
  tetradic_count2_3 = len(tetra_df2_3)





  #freqtable[abs(freqtable['avg_hue']-30) < 10].sort_values(by='freq', ascending = False)


  og_sat = float(freqtable[freqtable['name'] == color]['avg_saturation'])
  og_light = float(freqtable[freqtable['name'] == color]['avg_lightness'])
  xarr,yarr = curvecheck(og_sat,og_light)



  freqlist_arr = [[] for i in range(17)]


  #clock.tick()

  self_distance,freqlist_arr[0] = hue_distance_calc(color_hue, xarr, yarr, self_df)
  comp_distance,freqlist_arr[1] = hue_distance_calc(comphue, xarr, yarr, complementary_df)
  anal_distance1,freqlist_arr[2]= hue_distance_calc(analagous_hue1, xarr, yarr, anal_df1)
  anal_distance2 ,freqlist_arr[3]= hue_distance_calc(analagous_hue2, xarr, yarr, anal_df2)
  tri_distance1 ,freqlist_arr[4]= hue_distance_calc(triadic_hue1, xarr, yarr, tri_df1)
  tri_distance2 ,freqlist_arr[5]= hue_distance_calc(triadic_hue2, xarr, yarr, tri_df2)
  split_distance1 ,freqlist_arr[6]= hue_distance_calc(split_hue1, xarr, yarr, split_df1)
  split_distance2 ,freqlist_arr[7]= hue_distance_calc(split_hue2, xarr, yarr, split_df2)
  square_distance1 ,freqlist_arr[8]= hue_distance_calc(comphue, xarr, yarr, square_df1)
  square_distance2 ,freqlist_arr[9]= hue_distance_calc(square_hue1, xarr, yarr, square_df2)
  square_distance3,freqlist_arr[10] = hue_distance_calc(square_hue2, xarr, yarr, square_df3)
  tetra_distance1,freqlist_arr[11] = hue_distance_calc(comphue, xarr, yarr, tetra_df1)
  tetra_distance1_2,freqlist_arr[12] = hue_distance_calc(analagous_hue1, xarr, yarr, tetra_df1_2)
  tetra_distance1_3 ,freqlist_arr[13]= hue_distance_calc(split_hue1, xarr, yarr, tetra_df1_3)
  tetra_distance1,freqlist_arr[14] = hue_distance_calc(comphue, xarr, yarr, tetra_df1)
  tetra_distance2_2 ,freqlist_arr[15]= hue_distance_calc(analagous_hue2, xarr, yarr, tetra_df2_2)
  tetra_distance2_3,freqlist_arr[16] = hue_distance_calc(split_hue2, xarr, yarr, tetra_df2_3)

  #clock.tock()


  freqlist_arr = np.asarray(freqlist_arr).T
  columns = ["prim_color", "type", "subtype", "count", "count%", "avg_distance", "hue_arr","f1", "f2", "f3", "f4","f5","f6","f7","f8", "f9", "f10"]
  prim_color = np.asarray([color for _ in range(17)]).T
  typearr = np.asarray(["Self","Comp", "Analg", "Analg", "Tri", "Tri", "Split", "Split", "Square", "Square", "Square", "Tetra1", "Tetra1", "Tetra1", "Tetra2", "Tetra2", "Tetra2"]).T
  subtypearr = np.asarray(["", "", "-", "+", "-", "+", "+", "-", "comp", "+", "-", "comp", "-", "- comp", "comp", "+", "+ comp"]).T
  count = np.asarray([self_count, complementary_count,anal_count1,anal_count2, triadic_count1, triadic_count2, split_count1, split_count2, square_count1, square_count2, square_count3, tetradic_count1_1, tetradic_count1_2, tetradic_count1_3, tetradic_count2_1, tetradic_count2_2, tetradic_count2_3]).T
  count_perc = count/count.sum()
  avg_distance = np.asarray([self_distance, comp_distance, anal_distance1, anal_distance2, tri_distance1, tri_distance2, split_distance1, split_distance2, square_distance1, square_distance2, square_distance3,
                  tetra_distance1, tetra_distance1_2, tetra_distance1_3, tetra_distance1, tetra_distance2_2, tetra_distance2_3]).T

  hue_arr = [[color_hue], [comphue],[analagous_hue1, analagous_hue2],[analagous_hue1, analagous_hue2],
             [triadic_hue1, triadic_hue2],[triadic_hue1, triadic_hue2],[split_hue2, split_hue1],
             [split_hue2, split_hue1],[comphue, square_hue2, square_hue1],[comphue, square_hue2, square_hue1],
             [comphue, square_hue2, square_hue1], [comphue, analagous_hue1, split_hue1],[comphue, analagous_hue1, split_hue1],
             [comphue, analagous_hue1, split_hue1], [comphue, analagous_hue2, split_hue2],[comphue, analagous_hue2, split_hue2],
             [comphue, analagous_hue2, split_hue2]]

  distance_df = pd.DataFrame(columns = columns, data = np.asarray([prim_color, typearr, subtypearr, count, count_perc ,avg_distance,hue_arr,
                                                                  freqlist_arr[0], freqlist_arr[1], freqlist_arr[2], freqlist_arr[3],
                                                                  freqlist_arr[4], freqlist_arr[5], freqlist_arr[6], freqlist_arr[7],
                                                                  freqlist_arr[8], freqlist_arr[9]]).T)
  distance_df["count"] = pd.to_numeric(distance_df["count"])
  distance_df["avg_distance"] = pd.to_numeric(distance_df["avg_distance"])
  distance_df["count"] = pd.to_numeric(distance_df["count"])
  distance_df["f1"] = pd.to_numeric(distance_df["f1"])/distance_df["count"]
  distance_df["f2"] = pd.to_numeric(distance_df["f2"])/distance_df["count"]
  distance_df["f3"] = pd.to_numeric(distance_df["f3"])/distance_df["count"]
  distance_df["f4"] = pd.to_numeric(distance_df["f4"])/distance_df["count"]
  distance_df["f5"] = pd.to_numeric(distance_df["f5"])/distance_df["count"]
  distance_df["f6"] = pd.to_numeric(distance_df["f6"])/distance_df["count"]
  distance_df["f7"] = pd.to_numeric(distance_df["f7"])/distance_df["count"]
  distance_df["f8"] = pd.to_numeric(distance_df["f8"])/distance_df["count"]
  distance_df["f9"] = pd.to_numeric(distance_df["f9"])/distance_df["count"]
  distance_df["f10"] = pd.to_numeric(distance_df["f10"])/distance_df["count"]
  distance_df["avg_distance"] /= distance_df["count"]


  #clock.tock()


  k_exp = 1.4 #constant, balancing out the score
  score = distance_df["count"]*np.exp(distance_df["avg_distance"]*-1*k_exp)
  distance_df.insert(len(distance_df.columns), 'score',np.asarray(score))
  

  #clock.tock()



  return distance_df, color_hue, xarr, yarr 

In [ ]:
def construct_palette(pal_limit, ranking_df, distance_df, xarr, yarr, currtype, standard,color_df,palette_length=6):


  #print(distance_df[distance_df['type'] == currtype]['hue_arr'].iloc[0])
  hue_arr = list(distance_df[distance_df['type'] == currtype]['hue_arr'].iloc[0])
  
  #print(hue_arr)
  #print(type(hue_arr))
  #print(currtype)
  #print("----------------------")
  curr_df = distance_df[distance_df["type"] == currtype]
  num_colors = len(curr_df)
  if currtype == "Analg":
    minus_arr = curr_df[curr_df['subtype'] == "-"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    plus_arr = curr_df[curr_df['subtype'] == "+"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    ratio = float(curr_df[curr_df['subtype'] == "-"]['score'])/float(curr_df[curr_df['subtype'] == "+"]['score'])
    #print(curr_df[curr_df['subtype'] == "-"]['score'])
    #print(curr_df[curr_df['subtype'] == "+"]['score'])
    #print(ratio)
    if ratio > 2.5:
      #- is 30, + is 10
      #[analagous_hue1, analagous_hue2]
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", plus_arr)
    elif ratio < .4:
      #- is 10, + is 30
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
    else:
      #- is 30, + is 30
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
  elif currtype == "Tetra1":
    comp_arr = curr_df[curr_df['subtype'] == "comp"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    minus_arr = curr_df[curr_df['subtype'] == "-"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    minus_comp_arr = curr_df[curr_df['subtype'] == "-comp"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]

    if float(curr_df['score'].max()) == float(curr_df[curr_df['subtype'] == "comp"]['score']):
      #[comphue, analagous_hue1, split_hue1]
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", minus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "10", minus_comp_arr)
    elif float(curr_df['score'].max()) == float(curr_df[curr_df['subtype'] == "-"]['score']):
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", minus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "10", minus_comp_arr)
    else:
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", minus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "30", minus_comp_arr)
  elif currtype == "Tri":
    minus_arr = curr_df[curr_df['subtype'] == "-"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    plus_arr = curr_df[curr_df['subtype'] == "+"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    ratio = float(curr_df[curr_df['subtype'] == "-"]['score'])/float(curr_df[curr_df['subtype'] == "+"]['score'])
    if ratio > 2.5:
      #- is 30, + is 10
      #[triadic_hue1, triadic_hue2]
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", plus_arr)
    elif ratio < .4:
      #- is 10, + is 30
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
    else:
      #- is 30, + is 30
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
  elif currtype == "Square":
    comp_arr = curr_df[curr_df['subtype'] == "comp"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    minus_arr = curr_df[curr_df['subtype'] == "-"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    plus_arr = curr_df[curr_df['subtype'] == "+"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]

    if float(curr_df['score'].max()) == float(curr_df[curr_df['subtype'] == "comp"]['score']):
      #[comphue, square_hue2, square_hue1]
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", minus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "10", plus_arr)
    elif float(curr_df['score'].max()) == float(curr_df[curr_df['subtype'] == "-"]['score']):
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", minus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "10", plus_arr)
    else:
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", minus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "30", plus_arr)

  elif currtype == "Tetra2":
    comp_arr = curr_df[curr_df['subtype'] == "comp"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    plus_arr = curr_df[curr_df['subtype'] == "+"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    plus_comp_arr = curr_df[curr_df['subtype'] == "+comp"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]

    if float(curr_df['score'].max()) == float(curr_df[curr_df['subtype'] == "comp"]['score']):
      #[comphue, analagous_hue2, split_hue2]
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", plus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "10", plus_comp_arr)
    elif float(curr_df['score'].max()) == float(curr_df[curr_df['subtype'] == "+"]['score']):
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "10", plus_comp_arr)
    else:
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", comp_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", plus_arr)
      swath3 = swath_gen(hue_arr[2], xarr,yarr, "30", plus_comp_arr)

  elif currtype == "Split":
    minus_arr = curr_df[curr_df['subtype'] == "-"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    plus_arr = curr_df[curr_df['subtype'] == "+"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    ratio = float(curr_df[curr_df['subtype'] == "-"]['score'])/float(curr_df[curr_df['subtype'] == "+"]['score'])
    if ratio > 2.5:
      #- is 30, + is 10
      #[split_hue2, split_hue1]
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "10", plus_arr)
    elif ratio < .4:
      #- is 10, + is 30
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "10", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
    else:
      #- is 30, + is 30
      swath1 = swath_gen(hue_arr[0], xarr,yarr, "30", minus_arr)
      swath2 = swath_gen(hue_arr[1], xarr,yarr, "30", plus_arr)
  elif currtype == "Comp":
    #[comphue]
    rank_arr = curr_df[curr_df['type'] == "Comp"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']]
    swath1 = swath_gen(hue_arr[0], xarr,yarr, "60", rank_arr)
  else:
    print(f"Invalid type: {currtype} (check for spelling errors)")


  hexarr = list(standard)[0]
  for color_part in swath1:
    hexarr.append(tuple(convert_hsl_to_rgb(color_part)))
  if num_colors > 1:
    for color_part in swath2:
      hexarr.append(tuple(convert_hsl_to_rgb(color_part)))
  if num_colors > 2:
    for color_part in swath3:
      hexarr.append(tuple(convert_hsl_to_rgb(color_part)))

  

  #print(hexarr)
  #print(indices)
  #print(hexarr[indices])
  hexset = set(hexarr)
  hexarr = list(hexset)
  hexarr = np.asarray(hexarr)
  #print(hexarr)
  hexarr = np.asarray(n_mostcommon(color_df, hexarr, palette_length))

  indices = np.linspace(0,len(hexarr)-1, len(hexarr)).reshape(1,len(hexarr)).astype(int)
  #print(hexarr)
  plt.imshow(hexarr[indices])

  plt.axis("off")

  plt.show()

  print("\n\n")




In [ ]:
class ticktock:
  def __init__(self):
    self.totstart=  time.time()
    self.start = time.time()
    self.end = time.time()
  def tick(self):
    self.start = time.time()
    selfend = time.time()
  def tock(self, tocktick = True, print_time = True):
    self.end = time.time()
    timepassed = self.end-self.start
    if print_time: 
      print(f"{timepassed} seconds passed")
    if tocktick:
      self.start = time.time()
    return timepassed


In [ ]:
'''url = "https://www.pinterest.com/miraculouswip/posters/"
numsamples = 500
testcombo = assemble_df_direct(url, numsamples)
colornames, color_df = get_colordf(testcombo,0.4)
freqtable = get_freqtable(colornames,color_df, color_quant = 0.4)
freqtable.sort_values(by = 'freq', inplace = True, ascending = False)
name = colornames[0]
'''

'url = "https://www.pinterest.com/miraculouswip/posters/"\nnumsamples = 500\ntestcombo = assemble_df_direct(url, numsamples)\ncolornames, color_df = get_colordf(testcombo,0.4)\nfreqtable = get_freqtable(colornames,color_df, color_quant = 0.4)\nfreqtable.sort_values(by = \'freq\', inplace = True, ascending = False)\nname = colornames[0]\n'

In [ ]:


def pinterest_palette(url = 'https://www.pinterest.com/miraculouswip/granola-stuff/', numsamples = 1000,  color_quant = 0.4):
  clock = ticktock()
  
  
  print("fetching image files...")
  clock.tick()
  '''
  
  result = getImageFiles(url)
  clock.tock()
  
  print("assembling pixel counts...")
  testcombo = assemble_df(result[0], result[1], numsamples)

  clock.tock()
  '''
  testcombo = assemble_df_direct(url, numsamples)
  print(testcombo)
  clock.tock()
 
  print("filtering data...")
  colornames, color_df = get_colordf(testcombo,color_quant)

  clock.tock()

  freqtable = get_freqtable(colornames,color_df, color_quant = color_quant)
  freqtable.sort_values(by = 'freq', inplace = True, ascending = False)
  clock.tock()
  print("counting color frequency...")

  pal_limit=0.9
  distance_df_arr = []
  ranking_df_arr = []
  xy_arr = []
  clock.tock()
  for name in freqtable.head(10)["name"]:
    clock.tock()
    print(f"Computing color palettes for {name}")
    #print(f"|    {name}        |\n--------------------------")
    distance_df, color_hue, xarr, yarr = get_pallette(color_df, name, freqtable)
    clock.tock()
    print(f"Ranking {name}")
    ranking_df = get_rankingdf(distance_df, name)
    distance_df_arr.append(distance_df)
    ranking_df_arr.append(ranking_df)
    clock.tock()
    selflist = list(distance_df[distance_df["type"] == "Self"][["f1",'f2','f3','f4',"f5",'f6','f7','f8',"f9",'f10']].iloc[0])
    standard= swath_gen(color_hue, xarr, yarr, "60",selflist)
    for s in range(len(standard)):
      standard[s] = tuple(convert_hsl_to_rgb(standard[s]))
    
    xy_arr.append([name, xarr, yarr, standard])
    currtype = "Tri"
    #temporarily^
  

  tot_distancedf = pd.concat(distance_df_arr, sort = False)
  tot_rankingdf = pd.concat(ranking_df_arr, sort = False)

  clock.tock()
  print("Computing best palettes")
  tot_distancedf = tot_distancedf.merge(freqtable[['name', 'freq']], how='inner', left_on = "prim_color", right_on = "name")
  #total_score = tot_distancedf["score"]*tot_distancedf["freq"]

  tot_distancedf['freq'] = pd.to_numeric(tot_distancedf['freq']).astype(float)

  

  sqrtscore = np.sqrt(tot_distancedf['freq'])

  tot_score = sqrtscore*tot_distancedf['score']

  tot_distancedf.insert(len(tot_distancedf.columns), "tot_score",tot_score,True)

  xy_df = pd.DataFrame(columns = ["name", "xarr", "yarr", "standard"], data = xy_arr)


  tot_distancedf.sort_values(by='tot_score', inplace = True, ascending = False)

  iterdf = tot_distancedf[tot_distancedf["type"] != 'Self']
  iterdf.reset_index(inplace = True)

  #print(iterdf.index)


  clock.tock()
  for i in range(10):
    tempname = str(iterdf.iloc[i]["name"])
    temptype = iterdf.iloc[i]["type"]
    print(f"rank {i+1}:{tempname}, {temptype}        |\n--------------------------")
    construct_palette(pal_limit, iterdf[iterdf["prim_color"] == tempname], iterdf[iterdf["prim_color"] == tempname], xy_df[xy_df["name"] == tempname]["xarr"], xy_df[xy_df["name"] == tempname]["yarr"], temptype, xy_df[xy_df["name"] == tempname]["standard"], color_df)
    #print(iterdf.iloc[i]["name"])
  clock.tock()

In [ ]:

#add ur URL HERE!!!!

pinterest_palette('https://www.pinterest.com/miraculouswip/kattar-3adik/', numsamples = 500, color_quant = 0.22)
